# What is Byte-Pair Encoding (BPE)?

BPE is a tokenization algorithm widely used in modern NLP, particularly in large language models (LLMs), to break text into manageable units (tokens). Unlike traditional tokenization, which treats words or characters as tokens, BPE learns a vocabulary of tokens from a training corpus. These tokens can be words, subwords (like morphemes or smaller units), or even individual characters. The key advantage of BPE is its ability to handle unknown words by representing them as combinations of known subword units.

## Why is BPE Useful?

### Unknown Words
In NLP, models trained on one corpus (training data) may encounter unseen words in a test corpus. For example, if the training data has low, new, and newer but not lower, BPE ensures lower can be tokenized as low + er, allowing the model to process it.

### Flexibility
BPE creates a vocabulary that balances whole words and subword units, making it robust for various languages and text types.

### Efficiency
It reduces vocabulary size compared to character-level tokenization while maintaining the ability to represent any word.

# How Does BPE Work?

BPE consists of two main components:

1. **Token Learner**: Builds a vocabulary by iteratively merging the most frequent pairs of adjacent symbols in a training corpus.
2. **Token Segmenter**: Uses the learned vocabulary to tokenize new (test) text.

Here's a step-by-step explanation of the process, using the example from the book:

## 1. Token Learner

The token learner starts with a corpus and builds a vocabulary through iterative merging.

### Input Corpus (example from the book):

```text
5 low
2 lowest
6 newer
3 wider
2 new
```

Each word is split into individual characters, with a special end-of-word symbol (`</w>` in some implementations, but here represented as `_` for simplicity). The initial vocabulary is the set of all unique characters:

```text
Vocabulary: [, d, e, i, l, n, o, r, s, t, w]
```

### Steps:

- **Count Pairs**: Identify the most frequent pair of adjacent symbols in the corpus. For example, `e r` appears 9 times (6 in newer, 3 in wider).
- **Merge**: Combine the most frequent pair into a single token (e.g., `er`). Update the corpus by replacing all instances of `e r` with `er`.
- **Update Vocabulary**: Add the new token (`er`) to the vocabulary.
- **Repeat**: Continue merging the most frequent pair for a specified number of iterations (k), where k is a parameter.

### Example Merges (from the book):

1. Merge `e r` → `er` (9 occurrences). Vocabulary becomes: `[, d, e, i, l, n, o, r, s, t, w, er]`.
2. Merge `er *` → `er*` (word-final er). Vocabulary: `[, d, e, i, l, n, o, r, s, t, w, er, er_]`.
3. Merge `n e` → `ne`. Vocabulary: `[, d, e, i, l, n, o, r, s, t, w, er, er_, ne]`.
4. Continue merging (e.g., `ne w` → `new`, `l o` → `lo`, etc.) until k merges are complete.

### Result
The final vocabulary contains:
- Original characters.
- New tokens (subwords or full words) created by merging, like `er`, `ne`, `new`, `low`, `newer`.

## 2. Token Segmenter

The segmenter applies the learned vocabulary to tokenize new text (test data):

- Split the test text into individual characters.
- Apply the merges learned during training in the same order (greedy application).
- For each merge, replace the corresponding pair of tokens with the merged token if it exists in the vocabulary.

### Example:

**Test word: newer**
- Initial: `n e w e r`
- Apply merge 1: `n e w er` (since `e r` → `er`).
- Apply merge 2: `n e w er_` (since `er *` → `er*`).
- Apply merge 3: `ne w er_` (since `n e` → `ne`).
- Apply later merges: Eventually, `newer` becomes a single token `newer`.

**Test word: lower (unknown word)**
- Initial: `l o w e r`
- Apply merges: Results in `low er` (since `low` and `er` are in the vocabulary).

The segmenter ensures that even unknown words are tokenized into known subword units, solving the unknown word problem.

# What's Happening in BPE?

BPE is a way to tokenize text by learning a vocabulary from a training corpus (a collection of text). It starts with individual characters and iteratively combines the most frequent pairs of adjacent symbols into new tokens, building a vocabulary that includes both characters and subwords (or even full words). This vocabulary is then used to tokenize new text, ensuring even unknown words can be handled.

The process has two parts:
1. **Token Learner**: Learns the vocabulary by merging frequent pairs.
2. **Token Segmenter**: Uses the learned vocabulary to tokenize new text.

Let's dive into the example from the book to clarify each part.

## 1. Token Learner: Building the Vocabulary

The Token Learner takes a training corpus and creates a vocabulary by repeatedly merging the most frequent pairs of adjacent symbols.

### Input Corpus

The example corpus is:

```text
5 low
2 lowest
6 newer
3 wider
2 new
```

- **Meaning**: The word `low` appears 5 times, `lowest` 2 times, `newer` 6 times, `wider` 3 times, and `new` 2 times.
- **Step 1**: Split into Characters: Each word is broken into individual characters, with a special end-of-word symbol (`_` in this case, though some implementations use `</w>`). For example:
  - `low` → `l o w _`
  - `lowest` → `l o w e s t _`
  - `newer` → `n e w e r _`
  - `wider` → `w i d e r _`
  - `new` → `n e w _`

### Initial Vocabulary

All unique characters in the corpus:

```text
Vocabulary: [, d, e, i, l, n, o, r, s, t, w]
```

(The comma `,` represents the space or separator, but it's not used in merging here since merges happen within words.)

### How Merging Works

The algorithm:
1. Counts all pairs of adjacent symbols in the corpus.
2. Merges the most frequent pair into a single token.
3. Updates the corpus by replacing that pair with the new token.
4. Adds the new token to the vocabulary.
5. Repeats for a set number of merges (k).

Let's walk through the merges step-by-step using the example.

#### Merge 1: `e r` → `er`

**Count Pairs**: Look at all adjacent pairs in the corpus:
- In `newer` (`n e w e r *`): Pairs are `n e`, `e w`, `w e`, `e r`, `r *` (6 times, since newer appears 6 times).
- In `wider` (`w i d e r *`): Pairs are `w i`, `i d`, `d e`, `e r`, `r *` (3 times).
- Total counts: `e r` appears 6 (from newer) + 3 (from wider) = 9 times, the most frequent pair.

**Merge**: Replace `e r` with `er` in the corpus:
- `newer` → `n e w er _`
- `wider` → `w i d er _`

**Update Vocabulary**: Add `er`:

```text
Vocabulary: [, d, e, i, l, n, o, r, s, t, w, er]
```

#### Merge 2: `er *` → `er*`

**Count Pairs**: Recount pairs in the updated corpus:
- In `newer` (`n e w er *`): Pairs are `n e`, `e w`, `w er`, `er *` (6 times).
- In `wider` (`w i d er *`): Pairs are `w i`, `i d`, `d er`, `er *` (3 times).
- Total: `er _` appears 6 + 3 = 9 times, the most frequent.

**Merge**: Replace `er *` with `er*`:
- `newer` → `n e w er_`
- `wider` → `w i d er_`

**Update Vocabulary**:

```text
Vocabulary: [, d, e, i, l, n, o, r, s, t, w, er, er_]
```

#### Merge 3: `n e` → `ne`

**Count Pairs**:
- In `newer` (`n e w er_`): Pairs are `n e`, `e w`, `w er_` (6 times).
- In `new` (`n e w *`): Pairs are `n e`, `e w`, `w *` (2 times).
- Total: `n e` appears 6 + 2 = 8 times, the most frequent.

**Merge**: Replace `n e` with `ne`:
- `newer` → `ne w er_`
- `new` → `ne w _`

**Update Vocabulary**:

```text
Vocabulary: [, d, e, i, l, n, o, r, s, t, w, er, er_, ne]
```

#### Later Merges (summarized):

- Merge `ne w` → `new`: Combines `ne` and `w` to form `new`.
- Merge `l o` → `lo`: Combines `l` and `o`.
- Merge `lo w` → `low`: Forms the full word `low`.
- Merge `new er_` → `newer`: Forms the full word `newer`.
- And so on, until k merges are done.

### Final Vocabulary (after several merges):

```text
Vocabulary: [, d, e, i, l, n, o, r, s, t, w, er, er_, ne, new, lo, low, newer, ...]
```

- Contains original characters and new tokens (subwords like `er`, `ne`, or full words like `newer`, `low`).
- The number of merges (k) determines how many new tokens are added.

**Why This Matters**: The vocabulary now includes common subwords (`er`, `ne`) and words (`newer`, `low`), which can represent both known and unknown words in the test data.

## 2. Token Segmenter: Tokenizing New Text

The Token Segmenter:
- Takes a new (test) word.
- Splits it into individual characters.
- Applies the merges learned during training in the same order.
- Replaces pairs with merged tokens if they exist in the vocabulary.

### Example 1: Tokenizing `newer`

- **Input**: `newer` (a known word from the training corpus).
- **Step 1**: Split into Characters: `n e w e r _`.
- **Apply Merges** (in the order learned):
  - Merge 1: `e r` → `er`. So: `n e w er _`.
  - Merge 2: `er *` → `er*`. So: `n e w er_`.
  - Merge 3: `n e` → `ne`. So: `ne w er_`.
  - Later merge: `ne w` → `new`. So: `new er_`.
  - Later merge: `new er_` → `newer`. So: `newer`.
- **Result**: `newer` is tokenized as a single token `newer`, since it was learned as a full word.

### Example 2: Tokenizing `lower` (unknown word)

- **Input**: `lower` (not in training corpus).
- **Step 1**: Split into Characters: `l o w e r _`.
- **Apply Merges**:
  - Merge 1: `e r` → `er`. So: `l o w er _`.
  - Merge 2: `er *` → `er*`. So: `l o w er_`.
  - Merge 3: `n e` → `ne` (not applicable, no `n e` in lower).
  - Later merge: `l o` → `lo`. So: `lo w er_`.
  - Later merge: `lo w` → `low`. So: `low er_`.
  - No further merges apply (e.g., `low er_` isn't in the vocabulary).
- **Result**: `lower` is tokenized as `low er_` (two tokens).

**Key Point**: Even though `lower` wasn't in the training data, BPE tokenizes it using known subwords (`low` and `er_`), solving the unknown word problem.

## Visualizing the Process

Here's a simplified view of the corpus after a few merges:

### Initial Corpus:

```text
5 l o w _
2 l o w e s t _
6 n e w e r _
3 w i d e r _
2 n e w _
```

### After Merge 1 (`e r` → `er`):

```text
5 l o w _
2 l o w e s t _
6 n e w er _
3 w i d er _
2 n e w _
```

### After Merge 2 (`er *` → `er*`):

```text
5 l o w _
2 l o w e s t _
6 n e w er_
3 w i d er_
2 n e w _
```

### After Merge 3 (`n e` → `ne`):

```text
5 l o w _
2 l o w e s t _
6 ne w er_
3 w i d er_
2 ne w _
```

### Tokenizing `lower`:

- Start: `l o w e r _`
- After merges: `low er_` (two tokens).

# What is Word Normalization?

Word normalization is the process of transforming words or tokens into a standard format to ensure consistency in NLP tasks. The goal is to treat different surface forms of a word as the same when appropriate, improving generalization or matching in applications like search, speech recognition, or text analysis.

## Key Example: Case Folding

### Definition
Case folding is the simplest form of normalization, where all letters in a word are converted to lowercase (or sometimes uppercase).

### Example
The words `Woodchuck` and `woodchuck` are treated as identical after case folding to `woodchuck`.

### Why It's Useful

- Helps generalization in tasks like information retrieval (e.g., search engines) or speech recognition, where `Woodchuck` and `woodchuck` should match the same concept.
- Reduces vocabulary size by collapsing case variations.

### When It's Not Used

- In tasks like sentiment analysis, information extraction, or machine translation, case can carry meaning. For example:
  - `US` (the country) vs. `us` (the pronoun) have different meanings.
  - Keeping case distinctions can help models differentiate these terms.
- Some systems maintain both cased (preserving upper/lowercase) and uncased (all lowercase) versions of language models to balance flexibility.

## Other Normalization Examples

Beyond case folding, normalization may involve standardizing different spellings or formats of the same concept:

- **Example**: Normalizing `USA` and `US` to a single form (e.g., `US`) for consistency in information retrieval or extraction.
- **Example**: Standardizing `uh-huh` and `uhhuh` to a single form (e.g., `uhhuh`).
- **Trade-Off**: Normalization simplifies processing but may lose spelling-specific information (e.g., stylistic differences in `uh-huh` vs. `uhhuh`).

## Relation to BPE

- Systems using Byte-Pair Encoding (BPE) (from Section 2.5.2) often rely on subword tokenization and may not need further normalization, as BPE already handles variations by breaking words into subword units.
- Other NLP systems, however, may apply normalization (like case folding or standardizing US/USA) to ensure consistency before or after tokenization.

# Lemmatization

Lemmatization is a more advanced form of normalization that involves reducing different morphological forms of a word to a single base form, called the lemma. The lemma is the dictionary form or root of a word, ignoring inflectional variations (e.g., tense, number, case).

## Why Lemmatization?

### Purpose
Ensures that different forms of a word are treated as the same concept, especially in languages with complex morphology.

### Examples
- **English**: `am`, `are`, `is` → lemma `be`.
- **English**: `dinner`, `dinners` → lemma `dinner`.
- **Polish**: `Warszawa` (subject), `w Warszawie` (in Warsaw), `do Warszawy` (to Warsaw) → lemma `Warszawa`.

### Use Case
In web search, lemmatizing `woodchucks` to `woodchuck` ensures search results include pages with either form. In Polish, lemmatizing `Warszawie` to `Warszawa` helps retrieve all mentions of Warsaw regardless of grammatical form.

## How Lemmatization Works

### Morphological Parsing
Lemmatization often relies on analyzing a word's morphemes, the smallest meaning-bearing units in a language.

#### Types of Morphemes
- **Stem**: The core meaning-bearing part of a word (e.g., `walk` in `walking`).
- **Affixes**: Prefixes, suffixes, or other modifications (e.g., `-ing`, `-s`).

#### Example
The word `woodchucks` has:
- **Stem**: `woodchuck`.
- **Suffix**: `-s` (plural marker).
- **Lemma**: `woodchuck` (the singular, base form).

### Process
A lemmatizer uses a dictionary or morphological rules to map a word to its lemma. For example:
- **Input**: `He is reading detective stories.`
- **Lemmatized**: `He be read detective story` (each word is reduced to its lemma).

### Tools
Advanced lemmatizers use:
- **Morphological parsers**: Break down words into stems and affixes.
- **Dictionaries**: Map inflected forms to their lemmas (e.g., WordNet for English).
- **Rules**: Handle regular patterns (e.g., `-s` for plurals in English) or language-specific morphology (e.g., Polish case endings).

## Challenges in Lemmatization

- Requires knowledge of the language's morphology (e.g., English is simpler, but Polish has complex case systems).
- Needs context for ambiguous words (e.g., `saw` could be the lemma for `saw` (past tense of see) or `saw` (a cutting tool)).
- More computationally intensive than simple normalization like case folding.

# Stemming

Stemming is a simpler, cruder alternative to lemmatization for reducing words to a base form, called the stem. Unlike lemmatization, which produces a valid dictionary word (the lemma) using morphological analysis, stemming uses heuristic rules to strip off affixes (e.g., prefixes or suffixes), often resulting in a stem that may not be a valid word.

## What is Stemming?

### Definition
Stemming chops off word-final affixes (like `-s`, `-ing`) to reduce words to a common base form, ignoring complex morphological rules.

### Goal
Collapse different forms of a word to a single representation to improve generalization in tasks like information retrieval.

### Example
- `cats` → stem `cat` (removes `-s`).
- `motoring` → stem `motor` (removes `-ing`).
- `grasses` → stem `grass` (replaces `-sses` with `-ss`).

## The Porter Stemmer

The Porter Stemmer (Porter, 1980) is a classic stemming algorithm mentioned in the text. It's widely used due to its simplicity and effectiveness in English.

### How It Works

The Porter Stemmer applies a series of rewrite rules in sequential passes. Each rule transforms the word by removing or replacing affixes, and the output of one pass becomes the input for the next.

#### Sample Rules (from the text and the referenced website)
- `ATIONAL` → `ATE`: `relational` → `relate` (removes `-ational`, adds `-ate`).
- `ING` → `ε` (if the stem contains a vowel): `motoring` → `motor` (removes `-ing`).
- `SSES` → `SS`: `grasses` → `grass` (replaces `-sses` with `-ss`).

Rules are applied iteratively, and each rule checks conditions (e.g., the stem must contain a vowel for `ING` → `ε`).

### Example from the Text

**Input paragraph**:

```text
This was not the map we found in Billy Bones's chest, but an accurate copy, complete in all things—names and heights and soundings—with the single exception of the red crosses and the written notes.
```

**Stemmed output** (using the Porter Stemmer):

```text
Thi wa not the map we found in Billi Bone s chest but an accur copi complet in all thing name and height and sound with the singl except of the red cross and the written note
```

#### Observations
- `This` → `Thi` (over-stemming, removes `-s` incorrectly).
- `was` → `wa` (removes `-s`).
- `accurate` → `accur` (removes `-ate`).
- `copy` → `copi` (removes `-y`, possibly incorrect).
- `complete` → `complet` (removes `-e`).
- `things` → `thing` (removes `-s`).
- `names` → `name` (removes `-s`).
- `singl` → `singl` (from `single`, removes `-e`).
- Errors like `Thi` and `copi` show stemming's limitations (see below).

## Advantages of Stemming

- **Simplicity**: Faster and less resource-intensive than lemmatization, as it relies on rules rather than dictionaries or morphological parsers.
- **Generalization**: Collapses variants of a word (e.g., `running`, `runs` → `run`), useful for search engines where exact word forms are less important.

## Limitations of Stemming

- **Over-Generalization**: Incorrectly stems words, e.g., `policy` → `polic` (confused with `police`), as noted in the text.
- **Under-Generalization**: Fails to stem related forms, e.g., `European` → `Europe` (not recognized as related to `Europe`).
- **Non-Word Stems**: Produces stems that aren't valid words, e.g., `better` → `bett`, unlike lemmatization's valid lemmas.
- **Modern Usage**: Less common in modern NLP systems (e.g., those using BPE or advanced lemmatization) due to these errors, but still used in lightweight applications like search.

## Stemming vs. Lemmatization

### Lemmatization
- Uses linguistic knowledge (dictionaries, morphological parsers).
- Produces valid words (e.g., `running` → `run`, `better` → `good`).
- More accurate but computationally expensive.

### Stemming
- Uses simple rules, no linguistic knowledge.
- Produces stems that may not be words (e.g., `better` → `bett`).
- Faster but less precise, prone to errors.

### Example
- **Input**: `cats`, `running`, `better`.
- **Lemmatization**: `cat`, `run`, `good`.
- **Stemming (Porter)**: `cat`, `run`, `bett`.

# Sentence Segmentation

Sentence segmentation is the process of dividing a text into individual sentences, a critical step in NLP for tasks like machine translation, text summarization, or parsing, where sentences are processed as units.

## Why Sentence Segmentation?

- Many NLP systems operate on sentences, not raw text.
- Accurate sentence boundaries ensure correct input for downstream tasks (e.g., parsing, translation).
- Challenges arise due to ambiguous punctuation, particularly the period (`.`).

## Key Cues for Sentence Segmentation

### Punctuation Markers

#### Unambiguous
Question marks (`?`) and exclamation points (`!`) are clear sentence boundary indicators.

#### Ambiguous
Periods (`.`) can mark:
- **Sentence boundaries**: `I love NLP. It's fun.`
- **Abbreviations**: `Mr. Smith` (not a sentence end).
- **Both**: `The meeting is at Inc.` (period marks abbreviation and sentence end).

### Context
The role of a period depends on the surrounding text (e.g., `Inc.` vs. a sentence-ending period).

## How Sentence Segmentation Works

### General Approach

1. **Identify Periods**: Determine whether a period is part of a word (e.g., abbreviation like `Mr.`) or a sentence boundary.
2. **Use Rules or Machine Learning**:
   - **Rule-Based**: Apply hand-crafted rules, often integrated with word tokenization.
   - **Machine Learning**: Train models to classify periods as sentence boundaries or not (e.g., Kiss and Strunk, 2006).
3. **Abbreviation Dictionaries**: Use lists of common abbreviations (`Mr.`, `Inc.`, `Dr.`) to avoid misinterpreting periods.

### Stanford CoreNLP Example (from the text)

- Uses a rule-based approach tied to tokenization.
- A sentence ends when a punctuation mark (`.`, `!`, `?`) is not part of a token (e.g., not in `Mr.` or `3.14`).
- Optionally, sentences may end with closing quotes or brackets (e.g., `He said, "Hello."` → two sentences: `He said` and `Hello`).

### Joint Tokenization
Sentence and word tokenization are often done together, as periods affect both (e.g., deciding if `Inc.` is one token or ends a sentence).

## Challenges in Sentence Segmentation

- **Ambiguous Periods**: `Inc.` can be an abbreviation or a sentence end, requiring context to disambiguate.
- **Complex Cases**: Nested punctuation (e.g., `He said, "Stop!"`) or lists (e.g., `Items: a. Apple b. Banana`) complicates boundary detection.
- **Language-Specific Issues**: Some languages lack clear punctuation or use different conventions, requiring tailored approaches.

## Example

**Input text**:

```text
Mr. Smith went to Inc. He works there.
```

### Naive Approach
Split on every period → incorrect:
- `Mr.` (not a sentence).
- `Smith went to Inc.` (correct).
- `He works there.` (correct).

### Smart Approach (e.g., CoreNLP)
Recognize `Mr.` and `Inc.` as abbreviation tokens using a dictionary or rules.

**Output**:
- **Sentence 1**: `Mr. Smith went to Inc.`
- **Sentence 2**: `He works there.`

# Minimum Edit Distance

## What is Minimum Edit Distance?

Minimum edit distance is a measure of how different two strings are, defined as the minimum number of operations (insertions, deletions, or substitutions) needed to transform one string into another. It quantifies string similarity, which is useful in many NLP tasks where we need to compare or align text.

## Why is it Useful?

The text highlights three key applications:

### 1. Spelling Correction
- **Example**: A user types `graffe`. The system suggests `giraffe` (differs by one letter: a → i) over `grail` or `graf` (more differences) because it has a lower edit distance.

### 2. Coreference Resolution
- **Example**: Deciding if "Stanford Arizona Cactus Garden" and "Stanford University Arizona Cactus Garden" refer to the same entity. Their similarity (differing by one word) suggests they might be coreferent.

### 3. Speech Recognition Evaluation
- Compares a system's transcript to a reference transcript to measure accuracy. Fewer differences (lower edit distance) indicate better performance.

## Key Operations

The operations used to transform one string into another are:

- **Insertion**: Add a character (e.g., `cat` → `cast` by inserting `s`)
- **Deletion**: Remove a character (e.g., `cast` → `cat` by deleting `s`)
- **Substitution**: Replace one character with another (e.g., `cat` → `cut` by substituting `a` → `u`)

## Levenshtein Distance

The text describes two versions of the Levenshtein distance, a common edit distance metric:

### 1. Standard Levenshtein
- **Insertion**: Cost = 1
- **Deletion**: Cost = 1
- **Substitution**: Cost = 1 (0 if substituting a character with itself, e.g., `t` → `t`)

### 2. Alternative Levenshtein (used in the algorithm example)
- **Insertion**: Cost = 1
- **Deletion**: Cost = 1
- **Substitution**: Cost = 2 (equivalent to one insertion + one deletion; 0 if characters are identical)

### Example: Transform `intention` to `execution` (using alternative Levenshtein)
- **Operations**: Delete `i`, substitute `n` → `e`, substitute `t` → `x`, insert `u`, substitute `n` → `c`
- **Cost**: 1 (delete) + 2 (sub n → e) + 2 (sub t → x) + 1 (insert u) + 2 (sub n → c) = 8

## Alignment

An alignment visualizes how one string transforms into another by matching characters and indicating operations:

**Example** (from Fig. 2.14):

```text
INTE*NTION
||| |||||
*EXECUTION
d s s i s
```

- **d**: Delete I (no match in execution)
- **s**: Substitute N → E, T → X, N → C
- **i**: Insert U
- **Matches**: T, I, O, N align directly (cost = 0 for identical characters)

This alignment shows the operations needed to transform `intention` into `execution`.

## The Minimum Edit Distance Algorithm

Computing the minimum edit distance naively (trying all possible operation sequences) is inefficient due to the vast number of possible paths. Instead, the algorithm uses **dynamic programming**, a method that breaks a problem into smaller subproblems, stores their solutions, and combines them to solve the larger problem efficiently.

### Intuition of Dynamic Programming

- **Concept**: If you're transforming `intention` to `execution` via an intermediate string (e.g., `exention`), the optimal path to `exention` must be part of the overall optimal path. Otherwise, a shorter path to `exention` would imply a shorter overall path, contradicting optimality.
- **Approach**: Build a table (matrix) to store the edit distances for all prefixes of the source and target strings, computing larger distances from smaller ones.

### Formal Definition

- **Input**: Source string X of length n (e.g., `intention`), target string Y of length m (e.g., `execution`)
- **Goal**: Compute D[n, m], the minimum edit distance between X and Y
- **Subproblem**: D[i, j] is the minimum edit distance between the first i characters of X (X[1..i]) and the first j characters of Y (Y[1..j])

## Algorithm Steps

The algorithm (Fig. 2.17) uses a dynamic programming matrix D of size (n+1) × (m+1):

### 1. Initialization
- **D[0, 0] = 0** (empty source to empty target requires no operations)
- **First row (D[i, 0])**: Distance from X[1..i] to an empty string = i deletions
  - E.g., D[1, 0] = 1 (delete one character from X)
- **First column (D[0, j])**: Distance from empty string to Y[1..j] = j insertions
  - E.g., D[0, 1] = 1 (insert one character from Y)

### 2. Recurrence Relation
For each cell D[i, j], compute the minimum cost of transforming X[1..i] into Y[1..j] by considering three possible operations:

- **Deletion**: Remove X[i]. Cost = D[i-1, j] + del-cost(X[i])
- **Insertion**: Add Y[j]. Cost = D[i, j-1] + ins-cost(Y[j])
- **Substitution**: Replace X[i] with Y[j]. Cost = D[i-1, j-1] + sub-cost(X[i], Y[j])
  - If X[i] = Y[j], sub-cost = 0
  - If X[i] ≠ Y[j], sub-cost = 2 (in alternative Levenshtein)

**Formula (Eq. 2.24):**

```text
D[i, j] = min(
    D[i-1, j] + 1,              # Deletion
    D[i, j-1] + 1,              # Insertion
    D[i-1, j-1] + (2 if X[i] ≠ Y[j] else 0)  # Substitution
)
```

### 3. Termination
D[n, m] is the minimum edit distance.

## Example: `intention` → `execution`

- **Source (X)**: `intention` (n=9)
- **Target (Y)**: `execution` (m=9)
- **Matrix** (Fig. 2.18, partially reproduced):

```text
Src\Tar | # | e | x | e | c | u | t | i | o | n
--------|---|---|---|---|---|---|---|---|---|---
#       | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
i       | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 6 | 7 | 8
n       | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 7 | 8 | 7
t       | 3 | 4 | 5 | 6 | 7 | 8 | 7 | 8 | 9 | 8
e       | 4 | 3 | 4 | 5 | 6 | 7 | 8 | 9 |10 |11
n       | 5 | 4 | 5 | 6 | 7 | 8 | 9 |10 |11 |10
t       | 6 | 5 | 6 | 7 | 8 | 9 | 8 | 9 |10 |11
i       | 7 | 6 | 7 | 8 | 9 |10 | 9 | 8 | 9 |10
o       | 8 | 7 | 8 | 9 |10 |11 |10 | 9 | 8 | 9
n       | 9 | 8 | 9 |10 |11 |12 |11 |10 | 9 | 8
```

### Key Cells Analysis
- **D[0, 0] = 0** (empty to empty)
- **D[1, 0] = 1** (delete i from i)
- **D[0, 1] = 1** (insert e to empty)
- **D[1, 1]**: Compare i and e:
  - Delete i: D[0, 1] + 1 = 1 + 1 = 2
  - Insert e: D[1, 0] + 1 = 1 + 1 = 2
  - Substitute i → e: D[0, 0] + 2 = 0 + 2 = 2
  - D[1, 1] = min(2, 2, 2) = 2
- **D[9, 9] = 8** (final edit distance)

### Result
**Levenshtein distance = 8** (using cost of 2 for substitutions)